In [41]:
import pandas as pd
import ast

## DATASET 'Movies'

In [44]:
df_movies = pd.read_csv(r'C:\Users\GASTON\Desktop\PI1\movies_dataset.csv', low_memory=False)
#Como el archivo CSV es muy grande,con low_memory = False se permite a Pandas leerlo en bloques más pequeños 
# para mejorar el rendimiento

In [45]:
df_movies.head(3) #Observamos las columnas del dataset

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


## Desanidar columnas

In [46]:
#Se crea una función para normalizar las columnas anidadas con json normalize
def desanidarColumnas(df, columns):
    '''Esta función espera dos argumentos: el DataFrame a modificar, y una lista de columnas del DataFrame. 
    La función itera sobre esas columnas, convirtiéndolas en cadenas de texto utilizando literal_eval.
    Si la columna es nula, devuelve una lista vacía. 
    Expande las columna, y normaliza con json_normalize.
    Luego elimina la columna expandida. Retorna el DataFrame modificado.
    Requiere importar pandas y ast para funcionar.'''
    for column in columns:
        df[column] = df[column].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else [])
        # Expandir la columna en filas
        df = df.explode(column)
        # Normalizar la columna expandida
        col_df = pd.json_normalize(df[column])
        # Renombrar las columnas normalizadas para evitar superposiciones
        col_df = col_df.add_prefix(f'{column}')
        # Concatenar las columnas normalizadas con el DataFrame original
        df = df.drop(columns=[column]).reset_index(drop=True).join(col_df)
    return df

In [47]:
columnas = ['belongs_to_collection', 'production_companies', 'genres', 'production_countries', 'spoken_languages']
#Estas son las columnas anidadas

In [48]:
df_movies = desanidarColumnas(df_movies, columnas)

In [49]:
df_movies.head(2)

,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,...,vote_average,vote_count,production_companiesname,production_companiesid,genresid,genresname,production_countriesiso_3166_1,production_countriesname,spoken_languagesiso_639_1,spoken_languagesname
0,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,7.7,5415.0,Pixar Animation Studios,3.0,16.0,Animation,US,United States of America,en,English
1,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,7.7,5415.0,Pixar Animation Studios,3.0,35.0,Comedy,US,United States of America,en,English


In [50]:
#Se corrige la aplicación de ciertos cambios, utilizando strip para eliminar espacios en blanco en las columnas del DF.
def limpiezaColumnas(df):
    for col in df.select_dtypes(include=['object']):
        df[col] = df[col].apply(lambda x: x.strip() if isinstance(x, str) else x)
    return df

In [51]:
df_movies=limpiezaColumnas(df_movies)

## Tratamiento de Nulos
Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.

In [52]:
df_movies['revenue']= df_movies['revenue'].fillna(0) #Reemplazo los valores nulos con 0
df_movies['budget'] = df_movies['budget'].fillna(0) #Reemplazo los nulos con 0

Eliminar valores nulos del campo 'release_date'

In [53]:
df_movies.dropna(subset='release_date', inplace=True) #Se eliminan las filas con valores nulos en la columna 'release_date'.

De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde <br>
extraerán el año de la fecha de estreno.

In [54]:
print(type(df_movies['release_date'][1])) #Verificamos el formato de la columna con un registro de la misma
#En este caso, es un str.

<class 'str'>


In [55]:
df_movies['release_date'] = pd.to_datetime(df_movies['release_date'], format='%Y-%m-%d', errors='coerce') 
#Como hay valores que no son fechas, para manejarlos se usa el parámetro errors='coerce', para convertirlos en Nat (not a time). 
# En este mismo paso asignamos el formato de fecha. 

In [56]:
print("No son fechas:")
verif_fecha = df_movies[df_movies['release_date'].isna()] 
#En este paso identificamos si son elementos considerables. Se trata de 3 filas. Se pueden manejar individualmente,
#pero al tratarse de pocos registros en los que la mayoría de los valores son nulos, se decide eliminarlos.
verif_fecha

No son fechas:


,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,...,vote_average,vote_count,production_companiesname,production_companiesid,genresid,genresname,production_countriesiso_3166_1,production_countriesname,spoken_languagesiso_639_1,spoken_languagesname
353603,- Written by Ørnås,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Midnight Man,...,NaN,NaN,NaN,NaN,11176.0,Carousel Productions,NaN,NaN,NaN,NaN
353604,- Written by Ørnås,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Midnight Man,...,NaN,NaN,NaN,NaN,11602.0,Vision View Entertainment,NaN,NaN,NaN,NaN
353605,- Written by Ørnås,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Midnight Man,...,NaN,NaN,NaN,NaN,29812.0,Telescene Film Group Productions,NaN,NaN,NaN,NaN
471154,Rune Balot goes to a casino connected to the O...,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'iso_3166_1': 'US', 'name': 'United States o...",2012-09-29,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,NaN,Mardock Scramble: The Third Exhaust,...,NaN,NaN,NaN,NaN,2883.0,Aniplex,NaN,NaN,NaN,NaN
471155,Rune Balot goes to a casino connected to the O...,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'iso_3166_1': 'US', 'name': 'United States o...",2012-09-29,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,NaN,Mardock Scramble: The Third Exhaust,...,NaN,NaN,NaN,NaN,7759.0,GoHands,NaN,NaN,NaN,NaN
471156,Rune Balot goes to a casino connected to the O...,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'iso_3166_1': 'US', 'name': 'United States o...",2012-09-29,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,NaN,Mardock Scramble: The Third Exhaust,...,NaN,NaN,NaN,NaN,7760.0,BROSTA TV,NaN,NaN,NaN,NaN
471157,Rune Balot goes to a casino connected to the O...,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'iso_3166_1': 'US', 'name': 'United States o...",2012-09-29,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,NaN,Mardock Scramble: The Third Exhaust,...,NaN,NaN,NaN,NaN,7761.0,Mardock Scramble Production Committee,NaN,NaN,NaN,NaN
471158,Rune Balot goes to a casino connected to the O...,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'iso_3166_1': 'US', 'name': 'United States o...",2012-09-29,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,NaN,Mardock Scramble: The Third Exhaust,...,NaN,NaN,NaN,NaN,33751.0,Sentai Filmworks,NaN,NaN,NaN,NaN
516362,Avalanche Sharks tells the story of a bikini c...,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{'iso_3166_1': 'CA', 'name': 'Canada'}]",2014-01-01,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Beware Of Frost Bites,Avalanche Sharks,...,NaN,NaN,NaN,NaN,17161.0,Odyssey Media,NaN,NaN,NaN,NaN
516363,Avalanche Sharks tells the story of a bikini c...,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{'iso_3166_1': 'CA', 'name': 'Canada'}]",2014-01-01,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Beware Of Frost Bites,Avalanche Sharks,...,NaN,NaN,NaN,NaN,18012.0,Pulser Productions,NaN,NaN,NaN,NaN


In [57]:
df_movies = df_movies.dropna(subset=['release_date']) #Usamos dropna() para eliminar filas con valores nulos en release_date

In [61]:
# Convertir la columna 'release_date' a tipo datetime, manejando los errores
df_movies['release_date'] = pd.to_datetime(df_movies['release_date'], format='%Y-%m-%d', errors='coerce')

In [63]:
df_movies['release_year'] = df_movies['release_date'].dt.year #Aquí creamos la columna "release_year". 
#A la misma la completamos extrayendo el año de la columna 'release_date'

In [65]:
df_movies.reset_index(drop=True, inplace=True) # Se reinician los índices

## Se agregan métricas (ROI)

Se crea la columna RETURN, la cual contendrá el resultado de dividir revenue / budget.

In [81]:
df_movies['return'] = 0.0

In [82]:
df_movies['budget'] = pd.to_numeric(df_movies['budget'], errors='coerce')
df_movies['revenue'] = pd.to_numeric(df_movies['revenue'], errors='coerce')
#Es necesario convertir los valores a un tipo numérico (int o float) para que los datos tengan coherencia.
# errors=coerce convertirá las que tengan errores en NaN

In [83]:
df_movies.loc[(df_movies['budget'] > 0) & (df_movies['revenue'] > 0), 'return'] = df_movies['revenue'] / df_movies['budget']
#Se verifica que el valor de 'budget' y/o 'revenue' no sean 0. 
# Si lo son, la columna return conserva el valor '0' que le asignamos al crearla.
#Se utiliza .loc para asignar directamente a través de una selección condicional

In [86]:
df_movies['return'].isnull().sum() #Analizamos si hay valores nulos

0

## Eliminación de Columnas innecesarias

In [87]:
df_movies.columns

Index(['adult', 'budget', 'homepage', 'id', 'imdb_id', 'original_language',
       'original_title', 'overview', 'popularity', 'poster_path',
       'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title',
       'video', 'vote_average', 'vote_count', 'production_companiesname',
       'production_companiesid', 'genresid', 'genresname',
       'production_countriesiso_3166_1', 'production_countriesname',
       'spoken_languagesiso_639_1', 'spoken_languagesname', 'release_year',
       'return'],
      dtype='object')

In [88]:
#En este paso se eliminan las columnas que no se utilizarán
df_movies.drop(columns=['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage'], inplace=True)

In [89]:
#Observamos las nuevas columnas disponibles
df_movies.columns

Index(['budget', 'id', 'original_language', 'overview', 'popularity',
       'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'production_companiesname',
       'production_companiesid', 'genresid', 'genresname',
       'production_countriesiso_3166_1', 'production_countriesname',
       'spoken_languagesiso_639_1', 'spoken_languagesname', 'release_year',
       'return'],
      dtype='object')

## Tratamiento de Datos nulos, duplicados y otros valores

Una vez realizadas las primeras transformaciones, es preciso verificar: <br>
Filas duplicadas <br>
Las columnas de texto son de tipo string <br>
Unificar filas de acuerdo a su id

In [91]:
duplicadas = df_movies.duplicated().sum()
print(duplicadas)

163788


In [92]:
df_movies = df_movies.drop_duplicates() #Se eliminan las filas duplicadas

In [94]:
duplicadas = df_movies.duplicated().sum()
duplicadas #Verificamos nuevamente

0

In [101]:
df_movies.loc[:,'spoken_languagesname'] = df_movies['spoken_languagesname'].astype(str)
df_movies.loc[:,'genresname'] = df_movies['genresname'].astype(str)

In [103]:
# Agrupar por 'id' y agregar los idiomas y géneros
lang_genres = df_movies.groupby('id').agg({
    'spoken_languagesname': lambda x: ', '.join(set(x)), 
    'genresname': lambda x: ', '.join(set(x)) #Para cada columna, esta función lambda toma grupos de datos y los convierte en sets (elimina duplicados), y luego forma una cadena separadas por comas.
}).reset_index()


otras_col = ['title','budget', 'revenue', 'overview','release_date', 'vote_average', 'vote_count','popularity','runtime', 'status', 'tagline', 'production_companiesname', 'production_countriesname', 'release_year', 'return']
# Se guardan en esta variable las columnas con las que agruparé lang_gentes. Estas son columnas que no necesitan ser 
# agrupadas o agregadas, ya que tienen un solo valor por cada película. Si agrego la lista directamente en el groupby
# me arrojará error. Para solucionar esta cuestión, se utiliza la variable que guarda una lista de columnas.

col_valor_unico = df_movies.groupby('id')[otras_col].first().reset_index() #Para cada grupo de 'id', se selecciona el primer valor. Esto es porque cada 'id' tiene valores únicos para estas columnas.


movies = pd.merge(lang_genres, col_valor_unico, on='id', how='left')
#Se aplica merge para unir las columnas de valor único, con las de languages y genres. Esto devuelve un único DataFrame agrupado por id.

In [105]:
movies.head() #Se reducen las columnas considerablemente, y se han unificado correctamente los valores en género y lenguaje, que ahora son cadenas de texto.

,id,spoken_languagesname,genresname,title,budget,revenue,overview,release_date,vote_average,vote_count,popularity,runtime,status,tagline,production_companiesname,production_countriesname,release_year,return
0,100,English,"Crime, Comedy","Lock, Stock and Two Smoking Barrels",1350000.0,3897569.0,A card sharp and his unwillingly-enlisted frie...,1998-03-05,7.5,1671.0,4.60786,105.0,Released,A Disgrace to Criminals Everywhere.,Handmade Films Ltd.,United Kingdom,1998,2.887088
1,10000,Español,"Drama, Comedy",La estrategia del caracol,0.0,0.0,A group of tenants living in an old house are ...,1993-12-25,7.2,9.0,0.281609,116.0,Released,None,Ministère de la Culture et de la Francophonie,Colombia,1993,0.000000
2,10001,English,"Science Fiction, Comedy",Young Einstein,0.0,0.0,Albert Einstein is the son of a Tasmanian appl...,1988-12-15,4.5,46.0,2.562888,91.0,Released,E=mc²,Warner Bros.,Australia,1988,0.000000
3,100010,English,"Drama, War",Flight Command,0.0,0.0,"A rookie flyer, Ens. Alan Drake, joins the fam...",1940-12-27,6.0,1.0,0.769266,116.0,Released,"T-H-R-I-L-L AMERICA! Here come The Flying ""HEL...",Metro-Goldwyn-Mayer (MGM),United States of America,1940,0.000000
4,100017,Deutsch,Drama,Hounded,0.0,0.0,Deals with the obsessive relationship between ...,2006-08-06,4.8,7.0,2.964103,87.0,Released,None,MMM Film Zimmermann & Co,Germany,2006,0.000000


Con el nuevo DataFrame 'movies' corresponde ahora identificar y reemplazar valores nulos

In [108]:
#movies.isnull()
movies.isnull().sum()

id                              0
spoken_languagesname            0
genresname                      0
title                           0
budget                          0
revenue                         0
overview                      941
release_date                    0
vote_average                    0
vote_count                      0
popularity                      0
runtime                       246
status                         80
tagline                     24959
production_companiesname    11789
production_countriesname     6208
release_year                    0
return                          0
dtype: int64

In [109]:
#Crearemos una función para reemplazar valores nulos
def reemplazarNulos(df, columna, valor):
    """
    Esta función rellena los valores NaN en una columna específica de un DataFrame.
    Se esperan tres (3) parámentros:
    df (DataFrame): Sobre este DataFrame se realizará la operación.
    columna (str): Es el nombre de la columna del DataFrame en la que se reemplazarán los valores.
    valor: El valor con el que sr reemplazan los NaN en la columna.

    Se retorna un DataFrame con los nulos rellenados en las columnas indicadas.

    Si debo rellenar múltiples columnas con diferentes valores, se puede mapear un diccionario de claves-valor y aplicar la función de manera iterativa.
    """
    df[columna] = df[columna].fillna(valor)
    return df


In [110]:
mapeo_nulos = {
    'title': 'Sin Datos',
    'overview': 'Sin Datos',
    'release_date': '00/00/0000',
    'vote_average': 'Sin Datos',
    'vote_count': 'Sin Datos',
    'runtime': 'Sin Datos',
    'status': 'Sin Datos',
    'tagline': 'Sin Datos',
    'production_companiesname': 'Sin Datos',
    'production_countriesname': 'Sin Datos',
    'release_year': 0000
}
#Cuando no hay valores de fecha, se reemplazarán por 0.

for columna, dato in mapeo_nulos.items():
    movies = reemplazarNulos(movies, columna, dato) #Mediante un bucle for, aplico la función una única vez (para evitar utilizarla una vez por columna)


In [111]:
movies.isnull().sum() #Ya no tengo valores nulos

id                          0
spoken_languagesname        0
genresname                  0
title                       0
budget                      0
revenue                     0
overview                    0
release_date                0
vote_average                0
vote_count                  0
popularity                  0
runtime                     0
status                      0
tagline                     0
production_companiesname    0
production_countriesname    0
release_year                0
return                      0
dtype: int64

In [112]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45346 entries, 0 to 45345
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   id                        45346 non-null  object        
 1   spoken_languagesname      45346 non-null  object        
 2   genresname                45346 non-null  object        
 3   title                     45346 non-null  object        
 4   budget                    45346 non-null  float64       
 5   revenue                   45346 non-null  float64       
 6   overview                  45346 non-null  object        
 7   release_date              45346 non-null  datetime64[ns]
 8   vote_average              45346 non-null  float64       
 9   vote_count                45346 non-null  float64       
 10  popularity                45346 non-null  object        
 11  runtime                   45346 non-null  object        
 12  status            

In [115]:
movies['id'] = movies['id'].astype('int64')
movies['popularity'] = movies['popularity'].astype('float64')

In [118]:
#movies['popularity'].info()
#movies['id'].info()
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45346 entries, 0 to 45345
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   id                        45346 non-null  int64         
 1   spoken_languagesname      45346 non-null  object        
 2   genresname                45346 non-null  object        
 3   title                     45346 non-null  object        
 4   budget                    45346 non-null  float64       
 5   revenue                   45346 non-null  float64       
 6   overview                  45346 non-null  object        
 7   release_date              45346 non-null  datetime64[ns]
 8   vote_average              45346 non-null  float64       
 9   vote_count                45346 non-null  float64       
 10  popularity                45346 non-null  float64       
 11  runtime                   45346 non-null  object        
 12  status            

In [124]:
movies.to_csv(r'C:\Users\GASTON\Desktop\PI1\movies_transformado.csv') #Extraemos un archivo CSV con las transformaciones realizadas

## DATASET 'Credits'

In [119]:
df_credits = pd.read_csv(r'C:\Users\GASTON\Desktop\PI1\credits.csv', low_memory=False) #Creamos el DataFrame a partir del archivo CSV
df_credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


CAST

In [120]:

#Vamos a desdoblar en dos DataFrame diferentes para poder trabajar de mejor manera, aunque se repita dos veces el proceso.
#DF1: una copia a partir de cast e id. DF2, a partir de crew e id.
cast = df_credits[['cast', 'id']].copy()
crew = df_credits[['crew', 'id']].copy()

In [122]:
cast.head()

,cast,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...",11862


In [123]:
crew.head()

,crew,id
0,"[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [127]:
cast = desanidarColumnas(cast, ['cast'])
cast.head()

,id,castcast_id,castcharacter,castcredit_id,castgender,castid,castname,castorder,castprofile_path
0,862,14.0,Woody (voice),52fe4284c3a36847f8024f95,2.0,31.0,Tom Hanks,0.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg
1,862,15.0,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2.0,12898.0,Tim Allen,1.0,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg
2,862,16.0,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2.0,7167.0,Don Rickles,2.0,/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg
3,862,17.0,Slinky Dog (voice),52fe4284c3a36847f8024fa1,2.0,12899.0,Jim Varney,3.0,/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg
4,862,18.0,Rex (voice),52fe4284c3a36847f8024fa5,2.0,12900.0,Wallace Shawn,4.0,/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg


CREW

In [126]:
crew = desanidarColumnas(crew, ['crew'])
crew.head()

,id,crewcredit_id,crewdepartment,crewgender,crewid,crewjob,crewname,crewprofile_path
0,862,52fe4284c3a36847f8024f49,Directing,2.0,7879.0,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg
1,862,52fe4284c3a36847f8024f4f,Writing,2.0,12891.0,Screenplay,Joss Whedon,/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg
2,862,52fe4284c3a36847f8024f55,Writing,2.0,7.0,Screenplay,Andrew Stanton,/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg
3,862,52fe4284c3a36847f8024f5b,Writing,2.0,12892.0,Screenplay,Joel Cohen,/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg
4,862,52fe4284c3a36847f8024f61,Writing,0.0,12893.0,Screenplay,Alec Sokolow,/v79vlRYi94BZUQnkkyznbGUZLjT.jpg


De cada DataFrame, se seleccionan las columnas que se necesitarán para el desarrollo de la API

In [130]:
actores = cast[['id','castname']].copy().rename(columns={'castname':'actor'}) 
#Creamos una copia sólo con las columnas ID y actor (renombrando la columna castname)
actores = actores.reset_index(drop=True)
actores.head(3)

,id,actor
0,862,Tom Hanks
1,862,Tim Allen
2,862,Don Rickles


In [134]:
crewjobs = crew['crewjob'].unique()
crewjobs #Se puede observar que tengo muchos trabajos diferentes, pero sólo necesito el de 'Director'

array(['Director', 'Screenplay', 'Producer', 'Executive Producer',
       'Editor', 'Art Direction', 'Foley Editor', 'Animation',
       'ADR Editor', 'Orchestrator', 'Color Timer', 'CG Painter',
       'Original Story', 'Post Production Supervisor', 'Sculptor',
       'Animation Director', 'Music', 'Layout', 'Music Editor',
       'Negative Cutter', 'Title Designer',
       'Supervising Technical Director', 'Songs', 'Supervising Animator',
       'Sound Designer', 'Production Supervisor',
       'Executive Music Producer', 'Visual Effects Supervisor',
       'Visual Effects', 'Lighting Supervisor', 'Character Designer',
       'Set Dresser', 'Editorial Manager', 'Assistant Editor',
       'Editorial Coordinator', 'Production Coordinator',
       'Unit Publicist', 'Sound Re-Recording Mixer',
       'Supervising Sound Editor', 'Sound Effects Editor',
       'Sound Design Assistant', 'Assistant Sound Editor',
       'Casting Consultant', 'ADR Voice Casting',
       'Original Music Compos

In [137]:
directores = crew[crew['crewjob'] == 'Director'] #Se aplica una máscara para sólo obtener los valores en los que el crewjob sea 'Director'

directores = directores[['id', 'crewname']].copy().rename(columns={'crewname': 'director'})
# Filtramos las columnas de id (la usaremos para unir los DataFrame) y crewname, que se renombra a 'director'.
directores = directores.reset_index(drop=True)
directores.head(3)

,id,director
0,862,John Lasseter
1,8844,Joe Johnston
2,15602,Howard Deutch


In [139]:
crew[(crew['id'] == 5) & (crew['crewjob'] == 'Director')] #Esta es una verificación para confirmar películas con múltiples directores

,id,crewcredit_id,crewdepartment,crewgender,crewid,crewjob,crewname,crewprofile_path
400,5,52fe420dc3a36847f800012d,Directing,1.0,3110.0,Director,Allison Anders,/rsfxpcNEdmNqZyNcct0J695LhBK.jpg
401,5,52fe420dc3a36847f8000133,Directing,2.0,3111.0,Director,Alexandre Rockwell,None
402,5,52fe420dc3a36847f8000139,Directing,2.0,2294.0,Director,Robert Rodriguez,/bPqRdLWWwpOT8sBdj9PWOzNgwou.jpg
403,5,52fe420dc3a36847f800013f,Directing,2.0,138.0,Director,Quentin Tarantino,/9ci4NBvHXJktxjALdJsrtasqgtV.jpg


In [140]:
directores_actores = pd.merge(directores, actores, on='id', how='left')
#Se utiliza merge para unir por 'id' a los directores y actores
directores_actores.head()


,id,director,actor
0,862,John Lasseter,Tom Hanks
1,862,John Lasseter,Tim Allen
2,862,John Lasseter,Don Rickles
3,862,John Lasseter,Jim Varney
4,862,John Lasseter,Wallace Shawn


In [141]:
directores_actores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 615750 entries, 0 to 615749
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        615750 non-null  int64 
 1   director  615750 non-null  object
 2   actor     613363 non-null  object
dtypes: int64(1), object(2)
memory usage: 14.1+ MB


In [142]:
directores_actores['actor'] = directores_actores['actor'].astype(str)
directores_actores['director'] = directores_actores['director'].astype(str)
#Aquí simplemente se convierten las columnas en cadenas de texto

In [145]:
creditos = directores_actores.groupby('id').agg({
    'director': lambda x: ', '.join(set(x)) if len(set(x)) > 1 else x.iloc[0], #Aquí se verificará si hay más de un director después de convertirse en set (eliminándose duplicados). Si hay más de 1, se concatenan separados por comas. Si sólo hay uno, se agrega el primer valor.
    'actor': lambda x: ', '.join(x) #En este caso, siempre se concatenan los elementos.
}).reset_index() #Se concatenan por 'id' de película a los directores y actores
creditos.head()

,id,director,actor
0,2,Aki Kaurismäki,"Turo Pajala, Susanna Haavisto, Matti Pellonpää..."
1,3,Aki Kaurismäki,"Matti Pellonpää, Kati Outinen, Sakari Kuosmane..."
2,5,"Robert Rodriguez, Alexandre Rockwell, Allison ...","Tim Roth, Antonio Banderas, Jennifer Beals, Ma..."
3,6,Stephen Hopkins,"Emilio Estevez, Cuba Gooding Jr., Denis Leary,..."
4,11,George Lucas,"Mark Hamill, Harrison Ford, Carrie Fisher, Pet..."


In [146]:
creditos.duplicated().sum() #Verificamos que no hayan duplicados 

0

In [147]:
#Guardamos el DataFrame como CSV 
creditos.to_csv(r'C:\Users\GASTON\Desktop\PI1\credits_transformado.csv', index=False)

## UNIFICACION DE DATASETS

Para el desarrollo de la API, es preciso tener un Dataset unificado del que pueda obtener los diferentes valores para realizar consultas

In [148]:
#Se unifican los datasets a través de la columna 'id' 
movies_credits = pd.merge(movies, creditos, on='id', how='left')
movies_credits.head()

,id,spoken_languagesname,genresname,title,budget,revenue,overview,release_date,vote_average,vote_count,popularity,runtime,status,tagline,production_companiesname,production_countriesname,release_year,return,director,actor
0,100,English,"Crime, Comedy","Lock, Stock and Two Smoking Barrels",1350000.0,3897569.0,A card sharp and his unwillingly-enlisted frie...,1998-03-05,7.5,1671.0,4.607860,105.0,Released,A Disgrace to Criminals Everywhere.,Handmade Films Ltd.,United Kingdom,1998,2.887088,Guy Ritchie,"Jason Flemyng, Dexter Fletcher, Nick Moran, Ja..."
1,10000,Español,"Drama, Comedy",La estrategia del caracol,0.0,0.0,A group of tenants living in an old house are ...,1993-12-25,7.2,9.0,0.281609,116.0,Released,Sin Datos,Ministère de la Culture et de la Francophonie,Colombia,1993,0.000000,Sergio Cabrera,nan
2,10001,English,"Science Fiction, Comedy",Young Einstein,0.0,0.0,Albert Einstein is the son of a Tasmanian appl...,1988-12-15,4.5,46.0,2.562888,91.0,Released,E=mc²,Warner Bros.,Australia,1988,0.000000,Yahoo Serious,"Yahoo Serious, Odile Le Clezio, Peewee Wilson,..."
3,100010,English,"Drama, War",Flight Command,0.0,0.0,"A rookie flyer, Ens. Alan Drake, joins the fam...",1940-12-27,6.0,1.0,0.769266,116.0,Released,"T-H-R-I-L-L AMERICA! Here come The Flying ""HEL...",Metro-Goldwyn-Mayer (MGM),United States of America,1940,0.000000,Frank Borzage,"Robert Taylor, Ruth Hussey, Walter Pidgeon, Pa..."
4,100017,Deutsch,Drama,Hounded,0.0,0.0,Deals with the obsessive relationship between ...,2006-08-06,4.8,7.0,2.964103,87.0,Released,Sin Datos,MMM Film Zimmermann & Co,Germany,2006,0.000000,Angelina Maccarone,"Kostja Ullmann, Maren Kroymann, Moritz Grove, ..."


In [ ]:
#df_peliculas = pd.read_csv(r'C:\Users\GASTON\Desktop\PI1\movies_transformado.csv', low_memory=False)
#df_creditos = pd.read_csv(r'C:\Users\GASTON\Desktop\PI1\credits_transformado.csv', low_memory=False)
#movies_credits = pd.merge(df_peliculas, df_creditos, on='id', how='left')
#Si tengo los datasets con las transformaciones, puedo correr el código desde aquí para unificarlos.

In [150]:
#Guardamos el DataFrame unificado y transformado como CSV 
movies_credits.to_csv(r'C:\Users\GASTON\Desktop\PI1\movies_credits.csv', index=False)